In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pycbc.waveform import get_td_waveform
from pycbc.waveform.utils import phase_from_polarizations, amplitude_from_polarizations
from scipy.interpolate import InterpolatedUnivariateSpline as IUS

import lal
import lalsimulation as ls
import math
from scipy.constants import speed_of_light
from scipy.constants import gravitational_constant
from joblib import Parallel, delayed
from pprint import pprint

setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new


In [4]:
def func(case_number, mass_number):
    mass_switch = int(mass_number)
    case_num = int(case_number)
    lmax = 2
    casepath = ['/home/jakewilliams/Desktop/Mathematica/params_and_fpeak/M37.5_params.dat',
                    '/home/jakewilliams/Desktop/Mathematica/params_and_fpeak/M150_params.dat', '/home/jakewilliams/Desktop/Mathematica/params_and_fpeak/1000_random_params.dat'][mass_switch]
    with open(casepath, 'r') as f:
        # format: Case	f0	q	M_tot	chi1	chi2	theta1	phi1	theta2	phi2	f_peak
        data = np.loadtxt(f, skiprows=1)
        casel = data[:, 0]
        freqs = data[:, 1]
        Ml = data[:, 3]
        ql = data[:, 2]
        s1l = data[:, 4]
        s2l = data[:, 5]
        theta1l = data[:, 6]
        phi1l = data[:, 7]
        theta2l = data[:, 8]
        phi2l = data[:, 9]
        # f_peaks = data[:, 10]
    file_num = case_num - 1
    M=Ml[file_num]
    flow = freqs[file_num]
    f0_SI = flow
    qData = ql[file_num]
    if qData < 1:
        q = 1./qData
    else:
        q = qData
    m1 = M*q/(1. + q)
    m2 = m1/q
    s1 = s1l[file_num]
    s2 = s2l[file_num]
    s1angles = [theta1l[file_num], phi1l[file_num]]
    s2angles = [theta2l[file_num], phi2l[file_num]]
    s1x = s1*np.sin(s1angles[0])*np.cos(s1angles[1])
    s1y = s1*np.sin(s1angles[0])*np.sin(s1angles[1])
    s1z = s1*np.cos(s1angles[0])
    s2x = s2*np.sin(s2angles[0])*np.cos(s2angles[1])
    s2y = s2*np.sin(s2angles[0])*np.sin(s2angles[1])
    s2z = s2*np.cos(s2angles[0])
    chiA = [s1x, s1y, s1z]
    chiB = [s2x, s2y, s2z]

    lmax = 2
    modearray = ls.SimInspiralCreateModeArray()
    ls.SimInspiralModeArrayActivateMode(modearray, 2, 2)
    ls.SimInspiralModeArrayActivateMode(modearray, 2, 1)
    ls.SimInspiralModeArrayActivateMode(modearray, 2, -2)
    ls.SimInspiralModeArrayActivateMode(modearray, 2, -1)
    lal_params = lal.CreateDict()
    ls.SimInspiralWaveformParamsInsertModeArray(lal_params, modearray)
    hlm = ls.SimInspiralChooseTDModes(0., 1./(4*4096.), m1*lal.MSUN_SI, m2*lal.MSUN_SI,
                                      0, 0, s1z, 0, 0, s2z,
                                      f0_SI, f0_SI, 1e6*lal.PC_SI, lal_params, lmax, ls.NRSur7dq4)

    time_hI = ls.SphHarmTimeSeriesGetMode(
        hlm, 2, 2).deltaT * np.arange(len(ls.SphHarmTimeSeriesGetMode(hlm, 2, 2).data.data))

    hI = {}
    fI = {}
    hp_sum = 0.
    hc_sum = 0.
    hp_sum_22 = 0.
    hc_sum_22 = 0.
    modes = [(2, 2), (2, -2), (2, 1), (2, -1)]
    for lm in modes:
        hI[lm] = ls.SphHarmTimeSeriesGetMode(hlm, lm[0], lm[1]).data.data
        fI[lm] = IUS(time_hI, np.unwrap(
            np.angle(hI[lm]))).derivative()(time_hI)
        sYlm = lal.SpinWeightedSphericalHarmonic(
            0., lal.PI / 2. - 0., -2, lm[0], lm[1])
        if lm == (2, 2) or lm == (2, -2):
            hp_sum_22 += np.real(sYlm*hI[lm])
            hc_sum_22 += -np.imag(sYlm*hI[lm])
        hp_sum += np.real(sYlm*hI[lm])
        hc_sum += -np.imag(sYlm*hI[lm])
    time_hI = ls.SphHarmTimeSeriesGetMode(
        hlm, 2, 2).deltaT * np.arange(len(ls.SphHarmTimeSeriesGetMode(hlm, 2, 2).data.data))
    total = 0
    for lm in modes:
        total +=np.abs(ls.SphHarmTimeSeriesGetMode(
            hlm, lm[0], lm[1]).data.data)
    total = np.sqrt(total)
    freq22 = - \
        IUS(time_hI, np.unwrap(np.angle(hI[2, 2]))).derivative()(
            time_hI)/(2*np.pi)       
    return(case_num, freq22[np.argmax(total)])